# Data imbalance analysis
## Comparing the impact of dataset imbalance on classification performance metrics
In this notebook, we are going to explore how data imbalance affect classification scores on four different tasks, which have been chosen to exemplify typical use cases of ML analysis in neuroscience. To keep it simple, we will focus on binary classification problems (0 vs 1).

The four tasks are :
1. Synthetic data
2. EEG alpha oscillations (resting-state Eyes-Closed vs Eyes-Open)
3. MEG alpha oscillations (auditory vs visual stimulation)
4. MEG alpha oscillations (faces vs scrambled) 

In these tasks, we will observe the effect of data imbalance on 5 different performance metrics :   
1. Decoding Accuracy (Acc)  
2. Area Under the Curve (AUC)
3. F1
4. Balanced Accuracy (BAcc)

Finally, a few parameters of the classification pipeline must be kept in mind as they can also differentially impact performance on imbalanced data. Namely :
- Dataset size
- Classifier type
    - Support Vector Machine (SVM)
    - Linear Discriminant Analysis (LDA)
    - Logistic Regression (LR)
    - Random Forest (RF)
- Cross-validation scheme
    - K-Fold, k=5
    - Stratified K-Fold
    - Group K-Fold
    - Leave One Out (LOO)

## Imports
First, we start by importing functions from the provided toolbox as well as some useful plotting functions.

In [1]:
from imbalance.pipeline import Pipeline
from imbalance.viz import metric_balance, data_distribution
from imbalance.data import eegbci, gaussian_binary
from joblib import Parallel, delayed
import os
import numpy as np
import matplotlib.pyplot as plt
import string
from copy import deepcopy
import warnings
warnings.simplefilter("ignore", RuntimeWarning)

## Task 1 : Synthetic data
For that first classification task, we will generate data from two gaussian distributions, with means of 0 and 2.

In [ ]:
pls = []
#for distance in [0,1,3]:
    
def run(distance):
    # generate random data
    x, y, groups = gaussian_binary(n_samples_per_class=1000, n_groups=5, mean_distance=distance)
    # run the pipeline
    pl = Pipeline(
        x,
        y,
        groups,
        dataset_balance=np.linspace(0.1, 0.9, 5)[1:-1],
        classifiers=["lr", "lda", "svm", "rf"],
        metrics=["roc_auc", "accuracy", "f1", "balanced_accuracy"],
        n_permutations=0,
        n_init=10,
    )
    pl.evaluate()
    return deepcopy(pl)



pls = Parallel(n_jobs=3)(delayed(run)(dist) for dist in [0, 1, 3])
#pls.append(deepcopy(pl))

fitting classifiers:   0%|          | 0/480 [00:00<?, ?it/s, size=1, balance=0.3, classifier=LogisticRegression, metric=roc_auc]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   0%|          | 1/480 [00:00<00:11, 42.72it/s, size=1, balance=0.3, classifier=LogisticRegression, metric=accuracy]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   0%|          | 2/480 [00:00<00

fitting classifiers:   2%|▏         | 8/480 [00:00<00:08, 57.50it/s, size=1, balance=0.3, classifier=SVC, metric=roc_auc]                                 /home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   2%|▏         | 9/480 [00:00<00:10, 46.11it/s, size=1, balance=0.3, classifier=SVC, metric=accuracy]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   2%|▏         | 10/4

fitting classifiers:   3%|▎         | 15/480 [00:03<01:48,  4.28it/s, size=1, balance=0.3, classifier=RandomForestClassifier, metric=balanced_accuracy]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   3%|▎         | 14/480 [00:03<00:54,  8.53it/s, size=1, balance=0.3, classifier=RandomForestClassifier, metric=f1]      /home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   3%

fitting classifiers:   3%|▎         | 15/480 [00:05<02:26,  3.18it/s, size=1, balance=0.3, classifier=RandomForestClassifier, metric=balanced_accuracy]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   6%|▌         | 29/480 [00:06<01:12,  6.24it/s, size=1, balance=0.5, classifier=RandomForestClassifier, metric=accuracy]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   3%

fitting classifiers:   5%|▌         | 24/480 [00:06<01:59,  3.83it/s, size=1, balance=0.5, classifier=SVC, metric=roc_auc]                                 /home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   5%|▌         | 25/480 [00:06<01:58,  3.84it/s, size=1, balance=0.5, classifier=SVC, metric=accuracy]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   5%|▌         | 25

fitting classifiers:   8%|▊         | 40/480 [00:09<02:31,  2.90it/s, size=1, balance=0.7, classifier=SVC, metric=roc_auc]                                 /home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   6%|▌         | 29/480 [00:09<02:22,  3.16it/s, size=1, balance=0.5, classifier=RandomForestClassifier, metric=accuracy]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers: 

fitting classifiers:   8%|▊         | 40/480 [00:15<02:55,  2.51it/s, size=1, balance=0.7, classifier=SVC, metric=roc_auc]                                 /home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:   9%|▊         | 41/480 [00:15<02:17,  3.19it/s, size=1, balance=0.7, classifier=SVC, metric=accuracy]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:  13%|█▎        | 61

fitting classifiers:   9%|▉         | 44/480 [00:17<02:38,  2.75it/s, size=1, balance=0.7, classifier=RandomForestClassifier, metric=roc_auc]/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:  15%|█▌        | 72/480 [00:18<01:31,  4.46it/s, size=1, balance=0.5, classifier=SVC, metric=roc_auc]                                 /home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hyruuk/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting classifiers:  

In [ ]:
# visualize the result
fig, axes = plt.subplots(6, 3, figsize=(20, 30))
figtitle = "Synthetic data"
#fig.suptitle(figtitle, fontsize=25)
classifiers=["lr", "lda", "svm", "rf"]

show_leg_distrib=True
show_leg_metric=True
for ax_idx,ax in enumerate(axes.flat):
    ax.text(-0.1, 1.05, string.ascii_uppercase[ax_idx], transform=ax.transAxes, 
            size=20, weight='bold')
    if ax_idx in [0,1,2]:
        data_distribution(pls[ax_idx], ax=ax, show=False, show_leg=show_leg_distrib)
        show_leg_distrib=False
    elif ax_idx < 15:
        metric_balance(pls[ax_idx%3], ax=ax, show=False, classifier=classifiers[(ax_idx-3)//3], show_leg=show_leg_metric)
        show_leg_metric = False

## Task 2 : EEG analysis

In [ ]:
#pipeline_path="../imbalance/data/eeg.pickle"
features_path ="../imbalance/data/eeg_features.npy"

# load or generate dataset
if not os.path.isfile(features_path):
    x, y, groups = eegbci('../imbalance/data',roi=lambda x: x[0] in ['P','O'])
    np.save(features_path,dict(x=x, y=y, groups=groups))
else:
    features = np.load(features_path,allow_pickle=True).item()
    x, y, groups = features["x"] , features["y"] , features["groups"]

In [ ]:
pl = Pipeline(
    x,
    y,
    groups,
    dataset_balance=np.linspace(0.037, 1-0.037, 100)[1:-1],
    classifiers=['rf',"lda","svm","lr"],
    metrics=[ "roc_auc","accuracy", "f1", "balanced_accuracy"],
)
# fit and evaluate classifiers on dataset configurations
pl.evaluate()

In [ ]:
# visualize the result
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
figtitle = "EEG data"
#fig.suptitle(figtitle, fontsize=25)

for ax_idx,ax in enumerate(axes.flat):
    ax.text(-0.1, 1.05, string.ascii_uppercase[ax_idx], transform=ax.transAxes, 
            size=20, weight='bold')
    if ax_idx == 0:
        data_distribution(pl, ax=ax, show=False)
    else:
        metric_balance(pl, ax=ax, show=False)

## Task 3 : MEG 1

## Task 4 : MEG 2